In [5]:
import numpy as np
import requests
from functools import reduce
from numpy.random import default_rng


##### activation function: sigmoid
##### cost function: - correct * log (predicted)
## for delta calculation check: https://en.wikipedia.org/wiki/Delta_rule

In [2]:
NUM_HIDDEN_LAYERS = 8
def nonlin(x, deriv=False):
    if (deriv==True):
        return (x*(1-x))

    return 1/(1+np.exp(-x))

with open('words.txt', 'r') as f:
    lines = f.readlines()
    
f.close()

lines = [line[:len(line)-1] for line in lines]
revLines = [line[::-1] for line in lines]

# don't touch above this

In [3]:
max_len = 45 # pneumonoultramicroscopicsilicovolcanoconiosis

# make x and y columns

In [6]:
def makeXY():

    l = len(lines)
    X = np.empty([l, max_len])
    Y = np.empty([l, max_len])
    for ind, word in enumerate(lines):
        
        inArr = np.random.randint(1, 58, (1,max_len)) # ord('z') - 64  = 58
        inArrRev = np.random.randint(1, 58, (1,max_len)) 
        wordLen = len(word)
        for letIndx, let in enumerate(word):
            inArr[0, letIndx] = ord(let) - 64
            inArrRev[0, wordLen-1-letIndx] = ord(let) - 64
        # inArr[letIndx+1:] = 
        X[ind] = inArr
        Y[ind] = inArrRev
    return X, Y

X, Y = makeXY()
X_train = X[0:20000]
Y_train = Y[0:20000]
X_validation = X[20001:20]
X_test = X[20020:]
Y_test = X[20020:]

In [7]:
r = np.random.randint(1, 20000)
X[r], Y[r]

(array([ 8., 33., 36., 44., 37., 57., 51., 21., 56., 12.,  6., 35., 18.,
        21., 45., 22., 53., 51., 41., 37., 24., 55., 33., 34., 53., 22.,
        41., 27., 16., 10., 48., 12., 33., 42., 25., 42., 40., 47., 30.,
        55., 50., 43., 22., 52.,  6.]),
 array([57., 37., 44., 36., 33.,  8., 36., 37.,  4.,  2., 54., 50., 38.,
        22., 38., 21., 55., 42., 18., 12., 32.,  1., 30., 45., 47., 38.,
        30., 14., 36., 51., 57., 38., 15., 52., 11., 11., 14., 53., 57.,
        44., 54., 53., 47.,  4., 45.]))

# make connections between layers (synapses)

In [8]:
l = len(lines)
# seed is good for debugging, makes sure we start at
# the same time
np.random.seed(1)
sig = 0.7 # ratio between layers

# synapses (or connections)
# np.random.uniform(low=0.0, high=1.0, size=None)
rng = default_rng()



# the lower limit is set to 1.0 and upper is set to 26.1 because of the best and worst
# cases. In the best case the letter in position i might be the same letter as mid+i.
# In the worst case, the letter might an 'a' whereas it was supposed to be a 'z'
syn0 = rng.uniform(1.0, 26.1, (max_len, int(max_len*sig)))
syn1 = rng.uniform(1.0, 26.1, (int(max_len*sig), int(max_len/sig)))
syn2 = rng.uniform(1.0, 26.1, (int(max_len/sig), int(max_len)))
# synN = np.random.random((int(max_len*sig),max_len))     # this is going to be
import pdb

def forwardPropagation(x, y):
    # forward propagation
    l0 = x # 1 x max_len
    l1 = nonlin(np.dot(l0, syn0)) # 1 x normal . normal x small   
    l2 = nonlin(np.dot(l1, syn1)) # 1 x small . small x large
    l3 = nonlin(np.dot(l2, syn2)) # 1 x large . large x normal,   normal = max_len
    # l4 = nonlin(np.dot(l3, synN)) 
    
    finalLayer = l3
    return finalLayer

def calculate_loss(x, y):
    L = 0.0
    # For each sentence...
    N = len(y)
    pdb.set_trace()
    for i in np.arange(len(y)):
        o = forwardPropagation(x[i], y[i])
        # Add to the loss based on how off we were
        L += y[i] * (np.log(o))
    return -L / N
 
def bptt(x, y):
    T = len(x)
    # Perform forward propagation
    o = forward_propagation(x)
    # We accumulate the gradients in these variables
    # dLdi -> refers to delta for syni
    dLd0 = np.zeros(syn0.shape)
    dLd1 = np.zeros(syn1.shape)
    dLd2 = np.zeros(syn2.shape)
    delta_o = o
    delta_o[y] -= 1.
    # For each output backwards...
    dLdV += np.outer(delta_o, s[T-1].T)
    # Initial delta calculation
    delta_t = self.V.T.dot(delta_o) * (1 - (s[-1] ** 2))
    # Backpropagation through time (for at most self.bptt_truncate steps)
    for bptt_step in np.arange(max(0, (T-1)), T)[::-1]:
        #print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
        dLdW += np.outer(delta_t, s[bptt_step-1])
        try:
            X_j = model_vec[index_to_word[x[bptt_step]]]
        except KeyError:
            X_j = model_vec['unknown']
        dLdU += np.outer(delta_t, X_j)
        # Update delta for next step
        delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

# Performs one step of SGD.
def sgd_step(x, y, learning_rate):
    # Calculate the deltas
    # dLdi -> refers to delta for syni
    dLd0, dLd1, dLd2 = bptt(x, y)
    syn0 -= learning_rate * dLd0
    syn1 -= learning_rate * dLd1
    syn2 -= learning_rate * dLd2
    
# in prediction function, we will have to take the length of a given word, round the prediction to it's 
#       closest whole number and return the result in string format
NEPOCH = 10
losses = []
for _ in range(NEPOCH):
    # calculate loss
    loss = calculate_loss(X_train, Y_train)
#     losses.append(loss)
#     print(loss)
#     if (len(losses) > 1 and losses[-1] > losses[-2]):
#                 learning_rate = learning_rate * 0.5
#                 print ("Setting learning rate to %f" % learning_rate)
#     for i in range(len(y_train)):
#             # One SGD step
#             sgd_step(X_train[i], Y_train[i], learning_rate)
#             # num_examples_seen += 1

> <ipython-input-8-83d7fbd28eed>(36)calculate_loss()
     34     N = len(y)
     35     pdb.set_trace()
---> 36     for i in np.arange(len(y)):
     37         o = forwardPropagation(x[i], y[i])
     38         # Add to the loss based on how off we were

ipdb> b 20
Breakpoint 1 at <ipython-input-8-83d7fbd28eed>:20
ipdb> s
> <ipython-input-8-83d7fbd28eed>(37)calculate_loss()
     35     pdb.set_trace()
     36     for i in np.arange(len(y)):
---> 37         o = forwardPropagation(x[i], y[i])
     38         # Add to the loss based on how off we were
     39         L += y[i] * (np.log(o))

ipdb> s
--Call--
> <ipython-input-8-83d7fbd28eed>(20)forwardPropagation()
     18 import pdb
     19 
1--> 20 def forwardPropagation(x, y):
     21     # forward propagation
     22     l0 = x # 1 x max_len

ipdb> s
> <ipython-input-8-83d7fbd28eed>(22)forwardPropagation()
1    20 def forwardPropagation(x, y):
     21     # forward propagation
---> 22     l0 = x # 1 x max_len
     23     l1 = nonlin(np

ipdb> np.dot(l0, syn0)
array([14952.39071949, 16864.04585175, 17013.7703789 , 15410.9767617 ,
       15130.95097703, 18309.32070257, 14949.63260763, 15592.73884811,
       15338.7919841 , 15650.268388  , 15393.13198674, 14482.22446173,
       15203.68731098, 15666.6903115 , 16873.3450366 , 16381.16215405,
       15540.04831833, 15917.54981083, 17614.65945888, 13600.25185378,
       16835.70089185, 17300.12048938, 18388.99454109, 16582.37693017,
       16175.35083592, 13141.53011631, 16587.35500996, 17369.50174022,
       19056.75297062, 16075.62045642, 15221.14381898])
ipdb> nonlin(14952)
1.0
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0.]
> <ipython-input-8-83d7fbd28eed>(36)calculate_loss()
     34     N = len(y)
     35     pdb.set_trace()
---> 36     for i in np.arange(len(y)):
     37   

KeyboardInterrupt: 

In [47]:
help(np.random.uniform)

Help on built-in function uniform:

uniform(...) method of numpy.random.mtrand.RandomState instance
    uniform(low=0.0, high=1.0, size=None)
    
    Draw samples from a uniform distribution.
    
    Samples are uniformly distributed over the half-open interval
    ``[low, high)`` (includes low, but excludes high).  In other words,
    any value within the given interval is equally likely to be drawn
    by `uniform`.
    
    .. note::
        New code should use the ``uniform`` method of a ``default_rng()``
        instance instead; see `random-quick-start`.
    
    Parameters
    ----------
    low : float or array_like of floats, optional
        Lower boundary of the output interval.  All values generated will be
        greater than or equal to low.  The default value is 0.
    high : float or array_like of floats
        Upper boundary of the output interval.  All values generated will be
        less than or equal to high.  The default value is 1.0.
    size : int or tuple o

In [102]:
print(len(X), len(Y))

25487 25487


In [109]:
import logging
import gensim

In [111]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model_vec = gensim.models.KeyedVectors.load_word2vec_format('/Users/prajjwaldangal/Downloads/GoogleNews-vectors-negative300.bin', binary=True)
_VECTOR_SIZE = int(os.environ.get('VECTOR_SIZE', '300'))
_HIDDEN_DIM = int(os.environ.get('HIDDEN_DIM', '500'))
_LEARNING_RATE = float(os.environ.get('LEARNING_RATE', '0.0025'))
_NEPOCH = int(os.environ.get('NEPOCH', '10'))
evaluate_loss_after = 2
vocabulary_size = 25487

_MODEL_FILE = os.environ.get('MODEL_FILE')
vector_size = _VECTOR_SIZE
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
hidden_dim = _HIDDEN_DIM
learning_rate = _LEARNING_RATE
nepoch = _NEPOCH

2021-05-04 17:22:43,302 : INFO : loading projection weights from /Users/prajjwaldangal/Downloads/GoogleNews-vectors-negative300.bin
2021-05-04 17:23:29,733 : INFO : loaded (3000000, 300) matrix from /Users/prajjwaldangal/Downloads/GoogleNews-vectors-negative300.bin


Reading the input file


In [140]:
class RNNNumpy:
     
    def __init__(self, vector_dim , hidden_dim, label_dim = 6 , bptt_truncate = 6):
        # Assign instance variables
        self.label_dim = label_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.vector_dim = vector_dim
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./vector_dim), np.sqrt(1./vector_dim), (hidden_dim, vector_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (label_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

def forward_propagation(self, x):
    # import pdb; pdb.set_trace()

    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T+1, self.label_dim))
    # For each time step...
    for t in np.arange(T):
        X_i = x[t]
        s[t] = np.tanh(self.U.dot(X_i) + self.W.dot(s[t-1]))
    o = softmax(self.V.dot(s[-2]))
    return [o, s]
RNNNumpy.forward_propagation = forward_propagation

def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=0)
 
RNNNumpy.predict = predict


def calculate_total_loss(self, x, y):
    L = 0.0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[y[i]]
        # Add to the loss based on how off we were
        L += -1.0 * (np.log(correct_word_predictions))
    return L
 
def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    # y -> all y_train[i]
    N = len(y)
    return self.calculate_total_loss(x,y)/N
 
RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

def bptt(self, x, y):
    T = len(x)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[y] -= 1.
    # For each output backwards...
    dLdV += np.outer(delta_o, s[T-1].T)
    # Initial delta calculation
    delta_t = self.V.T.dot(delta_o) * (1 - (s[-1] ** 2))
    # Backpropagation through time (for at most self.bptt_truncate steps)
    for bptt_step in np.arange(max(0, (T-1)-self.bptt_truncate), (T-1)+1)[::-1]:
        #print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
        dLdW += np.outer(delta_t, s[bptt_step-1])     
        X_j = x[bptt_step]
        dLdU += np.outer(delta_t, X_j)   
        # Update delta for next step
        delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNNumpy.bptt = bptt


# Performs one step of SGD.
def numpy_sgd_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
 
RNNNumpy.sgd_step = numpy_sgd_step


# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate, nepoch, evaluate_loss_after):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    print ("There is going to be ", nepoch, " epoches")
    for epoch in range(nepoch):
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
            print ("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1
    filename11 = 'numpy_model.sav'
    pickle.dump(model, open(filename11, 'wb'))
model = RNNNumpy(vector_size,hidden_dim)

In [ ]:
train_with_sgd(model, X_train, y_train, learning_rate, nepoch, evaluate_loss_after)